**To use this notebook:** Run one line at a time waiting for each cell to return before running the next cell.

# Simple Output

SimpleOutput is a widget that provides an output area to display all types of output. 

It is designed to minimise the size of messages and/or number of messages sent to the frontend. It is not supposed to be a drop in replacement for the Ipywidget `Output' widget, rather it provides an alternate type of interface.

Compared to the Ipywidgets `Output` maintains a synchronised model of all loaded outputs. Each item added to `SimpleOutput` is serialized and sent to the frontend.  There is no representation of the data left on the Python side meaning that `SimpleOutput` is more suitable for logging applications. 

## Methods

There are two methods to add outputs 
1. `push`
2. `set`

and one '`clear`' to clear the outputs.


### `push`

`push` serializes and sends data as a custom message which is appended to the existing output.

In [ ]:
import anyio

import ipylab
from ipylab.simple_output import SimpleOutput

app = ipylab.App()

In [ ]:
so = SimpleOutput(layout={"max_height": "200px"})

In [ ]:
for i in range(50):
    so.push(f"test {i}\n")

In [ ]:
so

Or we could do it with one message...

In [ ]:
SimpleOutput(layout={"max_height": "200px"}).push(*(f"test {i}\n" for i in range(50)))

### Other formats are also supported

#### Ipython

In [ ]:
from IPython.display import HTML, Markdown

SimpleOutput().push(Markdown("## Markdown"), HTML("<h2>HTML</h2>"))

#### Ipywidgets

In [ ]:
import ipywidgets as ipw

SimpleOutput().push(ipw.Button(description="ipywidgets button"))

### set

`Set` is similar to push, but is run as task and clears the output prior at adding the new outputs. The task returns the number of outputs in use.

In [ ]:
so = SimpleOutput()
res = await so.set("Line one\n", "Line two")
so

In [ ]:
await anyio.sleep(0.1)
assert so.length == res  # noqa: S101
so.length

## max_continuous_streams and max_outputs

Notice that above the length is 1 even though we sent two values? 

This is because both items are streams, and by default they get put into the same output in the frontend. 

The maximum number of consecutive streams is configurable with `max_continuous_streams`.

In [ ]:
# Make each stream go into a new output.
so.max_continuous_streams = 0
res = await so.set("Line one\n", "Line two")
await anyio.sleep(0.1)
assert so.length == res  # noqa: S101
so.length

`max_outputs` limits the total number of outputs.

In [ ]:
so = SimpleOutput(max_continuous_streams=0, max_outputs=2)
so

In [ ]:
for i in range(100):
    await anyio.sleep(0.001)
    so.push(i)

# AutoScroll

AutoScroll is a widget that provides automatic scrolling around a content widget. It is intended to be used in panels placed in the shell, and doesn't work correctly when used in notebooks.

**Note**

Autoscroll uses a relatively new feature `onscrollend` ([detail](https://developer.mozilla.org/en-US/docs/Web/API/Element/scrollend_event)) and **may not work well on Safari** for fast update rates.

## Ipylab log viewer

The Ipylab log viewer uses a `SimpleOutput` with an `AutoScroll` widget to scroll its output.

In [ ]:
app.log_level = "DEBUG"
await app.commands.execute("Show log viewer")

In [ ]:
app.log.debug("Debug")
app.log.info("Info")
app.log.warning("Warning")
app.log.error("Error")
app.log.exception("Exception")
app.log.critical("Critical")

In [ ]:
app.shell.log_viewer.connections[0].close()

## Example usage

In [ ]:
from datetime import datetime

import anyio
import ipywidgets as ipw

import ipylab
from ipylab.simple_output import AutoScroll

app = await ipylab.App().ready()

In [ ]:
vb = ipw.VBox()
sw = AutoScroll(content=vb)
sw.sentinel_text = "sentinel"
sw_holder = ipw.VBox([sw], layout={"height": "200px", "border": "solid"})

enabled = ipw.Checkbox(description="Auto scroll", layout={"width": "120px"}, indent=False)
ipw.link((sw, "enabled"), (enabled, "value"))
sleep = ipw.FloatSlider(description="Sleep time (s)", value=0.3, min=0.05, max=1, step=0.01)

b_start = ipw.Button(description="Start", layout={"width": "max-content"})
b_clear = ipw.Button(description="Clear", layout={"width": "max-content"})
direction = ipw.RadioButtons(options=["fwd", "rev"], orientation="horizontal", layout={"width": "auto"})


def on_click(b):
    if b is b_start:
        if b.description == "Start":
            b.description = "Stop"

            async def generate_output():
                while b.description == "Stop":
                    vb.children = (*vb.children, ipw.HTML(f"It is now {datetime.now().isoformat()}"))  # noqa: DTZ005
                    await anyio.sleep(sleep.value)

            app.start_coro(generate_output())
        else:
            b.description = "Start"
    if b is b_clear:
        vb.children = ()


b_start.on_click(on_click)
b_clear.on_click(on_click)


def _observe_direction(_):
    if direction.value == "fwd":
        sw.mode = "end"
        vb.layout.flex_flow = "column"
    else:
        sw.mode = "start"
        vb.layout.flex_flow = "column-reverse"


direction.observe(_observe_direction, "value")

p = ipylab.Panel(
    [ipw.HBox([enabled, sleep, direction, b_start, b_clear], layout={"justify_content": "center"}), sw_holder]
)
await p.add_to_shell(mode=ipylab.InsertMode.split_right)

# Basic console example

In this example we create a basic console.

## Features (Provided by CodeEditor):
* await is allowed
* coroutines are awaited automatically
* Type hints
* Execution (Shift Enter)
* stdio captured during execution, but only output once execution completes
* History
* tooltips (documentation)

## Not implemented
* Ipython magic

In [ ]:
import io
import sys
from collections import deque
from contextlib import redirect_stdout
from typing import Self

import ipywidgets as ipw

import ipylab
from ipylab import Fixed
from ipylab.code_editor import CodeEditor
from ipylab.simple_output import AutoScroll, SimpleOutput
from ipylab.widgets import Panel


class SimpleConsole(Panel):
    prompt: Fixed[Self, CodeEditor] = Fixed(
        lambda _: CodeEditor(
            editor_options={"lineNumbers": False, "autoClosingBrackets": True, "highlightActiveLine": True},
            mime_type="text/x-python",
            layout={"flex": "0 0 auto"},
        ),
    )
    header: Fixed[Self, ipw.HBox] = Fixed(
        lambda c: ipw.HBox(
            children=(c["owner"].button_clear, c["owner"].autoscroll),
            layout={"flex": "0 0 auto"},
        ),
    )
    button_clear: Fixed[Self, ipw.Button] = Fixed(lambda _: ipw.Button(description="Clear", layout={"width": "auto"}))
    autoscroll: Fixed[Self, ipw.Checkbox] = Fixed(
        lambda _: ipw.Checkbox(description="Auto scroll", layout={"width": "auto"})
    )
    output: Fixed[Self, SimpleOutput] = Fixed(SimpleOutput)
    scroll: Fixed[Self, AutoScroll] = Fixed(lambda c: AutoScroll(content=c["owner"].output))
    history = Fixed(lambda _: deque(maxlen=100))

    def __init__(self, namespace_id: str, **kwgs):
        self.prompt.namespace_id = namespace_id
        super().__init__([self.header, self.scroll, self.prompt], **kwgs)
        self.button_clear.on_click(lambda _: self.output.push(clear=True))
        ipw.link((self.scroll, "enabled"), (self.autoscroll, "value"))
        self.title.label = "Simple console"
        self.prompt.evaluate = self.evaluate
        self.prompt.register_signal_callback("views.editorWidget.editor.edgeRequested", self.on_edge)

    async def evaluate(self, code: str):
        code = code or self.prompt.value
        try:
            f = io.StringIO()
            self.output.push(">>> " + code.replace("\n", "\n    ").strip() + "\n", stream_text=True)
            self.prompt.value = ""
            with redirect_stdout(f):
                result = await self.prompt.completer.evaluate(code)
                if isinstance(result, dict):
                    result = repr(result)
            if stdout := f.getvalue():
                self.output.push(stdout, stream_text=True)
            else:
                self.output.push(result)
        except Exception:
            text = self.app.logging_handler.formatter.formatException(sys.exc_info())  # type: ignore
            self.output.push({"output_type": "stream", "name": "stderr", "text": text})
        finally:
            self.history.append(code)

    def on_edge(self, data: ipylab.common.SignalCallbackData):
        if history := self.history:
            match data:
                case {"args": "top"}:
                    self.prompt.value = history[-1]
                    history.rotate()
                case {"args": "bottom"}:
                    history.reverse()
                    self.prompt.value = history[-1]

In [ ]:
sc = SimpleConsole("My namespace")
await sc.add_to_shell(mode=ipylab.InsertMode.split_bottom)

In [ ]:
sc2 = SimpleConsole("A separate namespace")
await sc2.add_to_shell(mode=ipylab.InsertMode.split_bottom)